# Project 1

TODO: add small description, imports and comments once the functions are written

In [ ]:
#import section


In [ ]:
# Linear regression using gradient descent
def least squares GD(y, tx, initial w,max_iters, gamma):
    

In [ ]:
#Linear regression using stochastic gradient descent
def least squares SGD(y, tx, initial w,max_iters, gamma):
    

In [ ]:
#Least squares regression using normal equations
def least squares(y, tx):
    

In [ ]:
#Ridge regression using normal equations
def ridge regression(y, tx, lambda_):
    

In [ ]:
#Logistic regression using gradient descent or SGD
def logistic regression(y, tx, initial w,max_iters, gamma):
    

In [ ]:
#Regularized logistic regression using gradient descent or SGD
def reg logistic regression(y, tx, lambda_ ,initial w, max_iters, gamma):
    